In [1]:
import os
import io
import json
from zipfile import ZipFile
import unicodedata
import nltk
from nltk.corpus import stopwords 
stop_words = set(stopwords.words('english'))

In [2]:
import re
import xapian
import pandas as pd
import numpy as np

In [3]:
pd.options.display.max_colwidth = 100

In [4]:
DBPATH = "ner_index"

In [5]:
if not os.path.exists(DBPATH):
    os.mkdir(DBPATH)

# Build Index

In [6]:
zf = ZipFile("wiki-pages-text.zip")

In [7]:
files = [item.filename for item in zf.filelist]
len(files), files[1]

(110, 'wiki-pages-text/wiki-009.txt')

In [8]:
def read_shard(zf, path):
    items = []
    fp = zf.open(path, mode='r')
    tfp = io.TextIOWrapper(fp)
    nlines = 0
    for line in tfp.readlines():
        nlines += 1
        line = unicodedata.normalize('NFD', line)
        match = re.match("(\S+)\s(\d+)\s(.*)\n", line)
        if match:
            items.append(match.groups())
        else:
            #print(line)
            pass
    fp.close()
    tfp.close()
    return items#, nlines

In [9]:
def read_shard_as_df(zf, path):
    items = read_shard(zf, path)
    raw_df = pd.DataFrame(data=items, columns=['doc_id', 'sentence', 'text'])
    func = lambda x: " ".join(x)
    return raw_df.groupby('doc_id')['text'].agg(func)

In [10]:
def preprocess_ner(s):
    #s = re.sub("^(The|the|A|a|An|an)\s", "", s)
    #s = re.sub("\s", "_", s) 
    return unicodedata.normalize('NFD', s)

def obtain_nouns(v):
    ents = {preprocess_ner(item['entity']) for item in v['named_entities']}
    ents_roots = {preprocess_ner(item['root']) for item in v['named_entities']}
    nps = {preprocess_ner(item['noun_phrase']) for item in v['noun_phrases']}
    nps_roots = {preprocess_ner(item['root']) for item in v['noun_phrases']}
    return ents.union(nps).union(ents_roots).union(nps_roots)

def read_ner_data(shard):
    df = pd.read_json('data/corpus/{}.json'.format(shard), orient="split")
    df.index.name = 'page_id'
    df.reset_index(inplace=True)
    for col in ['page_id']:
        df[col] = df[col].apply(lambda x: unicodedata.normalize('NFD', x))
    df.set_index('page_id', inplace=True)
    df['entities'] = df.parsed_text.apply(obtain_nouns)
    return df.entities.to_dict()

In [15]:
read_ner_data('001')

{u'1880\u201381_Heart_of_Midlothian_F.C._season': {u'Cup',
  u'Heart',
  u'Heart of Midlothian',
  u'Midlothian',
  u'Scottish',
  u'Season',
  u'Season 1880-81',
  u'a Scottish national level',
  u'level',
  u'season',
  u'sixth',
  u'the Scottish Cup',
  u'the sixth season',
  u'the sixth time',
  u'time'},
 u'1946_in_architecture': {u'The year',
  u'The year 1946',
  u'architecture',
  u'buildings',
  u'events',
  u'new buildings',
  u'some significant architectural events',
  u'year'},
 u'174_Phaedra': {u'174',
  u'Canadian',
  u'Canadian-American astronomer James Craig Watson',
  u'Greek',
  u'Greek mythology',
  u'James Craig Watson',
  u'Lightcurve',
  u'Lightcurve data',
  u'Phaedra',
  u'Photometric',
  u'Photometric observations',
  u'September',
  u'September 2, 1877',
  u'Watson',
  u'a rather irregular or elongated body',
  u'a sizable, rocky main belt asteroid',
  u'asteroid',
  u'body',
  u'data',
  u'mythology',
  u'observations',
  u'queen',
  u'the tragic lovelorn que

In [11]:
%%time
# Create or open the database we're going to be writing to.
db = xapian.WritableDatabase(DBPATH, xapian.DB_CREATE_OR_OPEN)

# Set up a TermGenerator that we'll use in indexing.
termgenerator = xapian.TermGenerator()
termgenerator.set_stemmer(xapian.Stem("en"))

for path in sorted(files[1:]):
    shard = ""
    m = re.match(".*wiki-(\d+).*", path)
    if m: shard = m[1]
    print(path)
    
    documents_df = read_shard_as_df(zf, path)
    keywords_dict = read_ner_data(shard)
    for doc_id, text in documents_df.items():
        try:
            # We make a document and tell the term generator to use this.
            doc = xapian.Document()
            termgenerator.set_document(doc)

            # Index fields without prefixes for general search.
            termgenerator.index_text(text)
            termgenerator.increase_termpos()

            # We use the identifier to ensure each object ends up in the
            # database only once no matter how many times we run the
            # indexer.
            idterm = u"Q" + doc_id
            doc.add_boolean_term(idterm)

            # Indexing keywords
            keywords = []
            if doc_id in keywords_dict:
                keywords = list(keywords_dict[doc_id])
            else:
                print("No keywords found for page_id={}".format(doc_id))
                
            for item in keywords_dict[doc_id]:
                doc.add_term(u"K" + item.lower())
                #doc.add_term(u"K:" + item)

            # save additional data
            data = dict(
                doc_id = doc_id,
                shard = shard,
                text = text,
                keywords = keywords,
            )
            doc.set_data(json.dumps(data))

            db.replace_document(idterm, doc)
        except Exception as e:
            print(doc_id, text, e)
db.commit()
db.close()

TypeError: '_sre.SRE_Match' object has no attribute '__getitem__'

In [12]:
!xapian-delve $DBPATH

UUID = 264d9bea-55c9-4caf-9979-abe630f0809a
number of documents = 5365408
average document length = 178.465
document length lower bound = 1
document length upper bound = 68701
highest document id ever used = 5365408
has positional information = true
revision = 537
currently open for writing = false


In [13]:
!xapian-delve -r 3129 -d $DBPATH

Data for record #3129:
{"doc_id": "11th_Gemini_Awards", "shard": "001", "text": "The 11th Gemini Awards was held on June 6 , 1997 , to honour achievements in Canadian television . It was hosted by Albert Schultz , and was broadcast on CBC .", "keywords": ["Awards", "television", "June 6, 1997", "Gemini Awards", "Canadian", "11th", "Canadian television", "The 11th Gemini Awards", "achievements", "June"]}
Term List for record #3129: 11th 1997 6 K11th Kachievements Kawards Kcanadian Kcanadian television Kgemini awards Kjune Kjune 6, 1997 Ktelevision Kthe 11th gemini awards Q11th_Gemini_Awards Zachiev Zalbert Zand Zaward Zbroadcast Zby Zcanadian Zcbc Zgemini Zheld Zhonour Zhost Zin Zit Zjune Zon Zschultz Ztelevis Zthe Zto Zwas achievements albert and awards broadcast by canadian cbc gemini held honour hosted in it june on schultz television the to was
